In [ ]:
import sys
sys.path.append('..')

from transformers import (
    AutoTokenizer,
)
from tqdm import tqdm

from utils import get_dataset

In [ ]:
model_id = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenize = lambda input: tokenizer(input["text"], truncation=True, padding=False, max_length=512)
dataset = get_dataset()
dataset = dataset.map(tokenize, batched=True)
dataset

In [ ]:
unique = "unique"
uknown = 'uknown'
info = {f"{k}_total": 0 for k in dataset}
info[unique] = set()
info[uknown] = 0

for split in dataset:
    totals = f"{split}_total"
    for tokens in tqdm(dataset[split]['input_ids'], desc=split):
        info[totals] += len(tokens)
        info[unique] |= set(tokens)
        info[uknown] += tokens.count(tokenizer.unk_token_id)

info[unique] = len(info[unique])
info[uknown] = info[uknown] * 100 / (info['train_total'] + info['test_total'] + info['validation_total'])
info